In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janestreet/data.parquet


This notebook's approach is using Neural Network.

* Ver 2: NN
* Ver 3: NN + PCA

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings ("ignore")
from tensorflow.keras.callbacks import EarlyStopping

import gc  

import tensorflow as tf
SEED = 1111
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [3]:
# this data is already excluded day <= 85

data = pd.read_parquet('../input/janestreet/data.parquet')
data.head()

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,86,0.859516,-0.003656,-0.005449,-0.017403,-0.028896,-0.021435,1,3.151305,5.467693,-0.164505,-0.189219,0.663966,0.988896,NaN,NaN,2.184804,3.278742,NaN,NaN,1.471544,2.391430,1.640887,3.938759,NaN,NaN,2.361346,4.711640,NaN,NaN,1.958027,4.069699,2.535238,4.813858,NaN,NaN,-0.194392,-0.336857,NaN,NaN,-0.073242,-0.131142,-0.197839,-0.288336,0.341815,0.599994,-0.202268,-0.471068,-0.405654,0.052440,-1.349263,-0.633717,-0.796947,-0.381209,-0.542896,-0.166690,0.222588,-0.554461,-0.187021,-0.551972,2.589171,4.141244,NaN,-0.414845,-1.150279,-0.908903,-1.341168,-1.325681,-1.110222,-1.656347,-1.810199,-4.434427,-1.661965,-1.101438,-2.136155,-2.048593,2.649794,3.595955,4.039769,NaN,-1.254960,NaN,-0.763423,-1.204365,0.649855,NaN,-0.374423,NaN,-0.335769,-0.345847,0.386626,NaN,-1.238212,NaN,-2.321156,-1.641860,-2.060506,NaN,-1.515613,NaN,-1.746285,-1.086886,-3.388070,NaN,1.607253,NaN,0.199793,0.969663,-2.434601,NaN,0.227364,NaN,-0.633981,-0.122468,-4.349793,NaN,0.322244,NaN,-0.458309,-0.032740,-3.018269,NaN,-1.219454,NaN,-2.608786,-1.611309,-2.724954,NaN,NaN,2.433699,4.282284,1.621115,4.331030,2.553220,3.799011,2.642943,3.998054,527894
1,86,0.000000,-0.009107,-0.013542,-0.022222,-0.032522,-0.026394,1,2.249176,2.618401,-0.304355,-0.276975,-0.035921,-0.036215,NaN,NaN,3.354857,3.040463,NaN,NaN,2.365050,2.376956,2.337125,3.438553,NaN,NaN,3.041641,4.165903,NaN,NaN,2.889146,4.174374,3.234317,4.276899,NaN,NaN,-1.644735,-2.479335,NaN,NaN,-1.321317,-1.491122,-2.478752,-2.496164,0.396227,0.435508,-0.248213,-0.439213,-0.993568,3.075146,0.017041,-1.907786,-1.643755,-0.873190,-0.730260,-0.627203,-0.498971,-2.203055,-1.725478,-2.086345,2.184227,-0.152785,NaN,-0.687010,-1.775500,-1.508644,-1.546806,-1.248566,-1.045781,-1.892714,-2.131570,-4.407797,-1.720470,-1.133543,-2.126616,-2.039426,3.591019,1.079669,3.550142,NaN,-1.871530,NaN,-1.154754,-1.803521,-0.008046,NaN,-0.198914,NaN,-0.167805,-0.182515,0.164888,NaN,-1.238212,NaN,-2.321156,-1.641860,-2.579694,NaN,-1.515613,NaN,-1.746285,-1.086886,-4.781603,NaN,2.557578,NaN,0.665543,1.704761,-1.965635,NaN,-0.079505,NaN,-0.857492,-0.512759,-4.546557,NaN,1.275872,NaN,-0.054892,0.872509,-3.120828,NaN,-1.881751,NaN,-3.280218,-2.261787,-3.617442,NaN,NaN,2.053416,-0.493276,1.661974,-1.082122,2.427706,-0.756115,2.210572,-0.639075,527895
2,86,0.590949,0.000347,-0.000376,-0.004051,-0.007995,-0.004743,-1,-0.365888,0.824004,-0.293208,-0.416391,-0.599185,-0.997330,NaN,NaN,-0.869330,0.174646,NaN,NaN,-2.376733,-2.602154,-0.580833,0.145479,NaN,NaN,-0.440224

In [4]:
# Select trade with weight !=  0:
data = data[data['weight'] != 0]
# # limit memory use: we change datatype from float64 to float32
data = data.astype({c: np.float32 for c in data.select_dtypes(include='float64').columns}) 

# create target variable
data['action'] = (data['resp'] > 0)*1

# fill null values with mean of each feature
data.fillna(data.mean(),inplace=True)

#create fetures list
features = [c for c in data.columns if 'feature' in c]

In [5]:
data.info()
data.isna().any().any()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571415 entries, 0 to 1862595
Columns: 139 entries, date to action
dtypes: float32(135), int64(4)
memory usage: 869.2 MB


False

In [6]:
# 20 / 80 split
df_trainvalid, df_test = np.split(data, [int(.8*len(data))])

df_train, df_valid = np.split(df_trainvalid, [int(.9*len(df_trainvalid))])

X_train = df_train[features]
y_train = df_train['action']

X_valid = df_valid[features]
y_valid = df_valid['action']

X_test = df_test[features]
y_test = df_test['action']


In [7]:
NORMALIZE_NONE = 0
NORMALIZE_MIN_MAX = 1
NORMALIZE_MEAN = 2


In [8]:
def normalize_data(df):
    if NORMALIZE_TYPE == NORMALIZE_MIN_MAX:
        return (df-df.min())/(df.max()-df.min())
    elif NORMALIZE_TYPE == NORMALIZE_MEAN:
        return (df-df.mean())/df.std()
    else:
        return df;


In [9]:
NORMALIZE_TYPE = NORMALIZE_MEAN

X_train = normalize_data(X_train).values
X_valid = normalize_data(X_valid).values
X_test = normalize_data(X_test).values



In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50).fit(X_train)
X_train = pca.transform(X_train)
X_valid = pca.transform(X_valid)
X_test = pca.transform(X_test)


In [11]:
del data, df_train, df_valid, df_trainvalid
gc.collect()

42

In [12]:
batch_size = 4096
step_train = len(X_train)//batch_size
step_valid = len(X_valid)//batch_size


# Create train, valid, test data with batch size 
train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).repeat()
val = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(batch_size).repeat()
test = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).repeat()



In [13]:
del X_train, y_train, X_valid, y_valid, y_test
gc.collect()


20

In [14]:
l_r = 1e-4

# A Sequential model

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(50,)),
    
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1),
    tf.keras.layers.Activation("sigmoid")

])


model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate = l_r),metrics=["AUC"])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               6528      
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0

In [15]:
gc.collect()

378

In [16]:
model.fit(train, validation_data=val, epochs=30, steps_per_epoch = step_train, validation_steps = step_valid, verbose = 1,callbacks = [
                EarlyStopping(monitor='loss', verbose=1, patience=10),
                EarlyStopping(monitor='val_loss', verbose=1, patience=10)
          ])



Epoch 1/30
276/276 [==============================] - 6s 14ms/step - loss: 0.7528 - auc: 0.5025 - val_loss: 0.6981 - val_auc: 0.5095
Epoch 2/30
276/276 [==============================] - 3s 11ms/step - loss: 0.7230 - auc: 0.5060 - val_loss: 0.6965 - val_auc: 0.5101
Epoch 3/30
276/276 [==============================] - 3s 11ms/step - loss: 0.7148 - auc: 0.5066 - val_loss: 0.6956 - val_auc: 0.5098
Epoch 4/30
276/276 [==============================] - 3s 12ms/step - loss: 0.7092 - auc: 0.5085 - val_loss: 0.6949 - val_auc: 0.5100
Epoch 5/30
276/276 [==============================] - 3s 12ms/step - loss: 0.7054 - auc: 0.5092 - val_loss: 0.6947 - val_auc: 0.5096
Epoch 6/30
276/276 [==============================] - 3s 11ms/step - loss: 0.7027 - auc: 0.5094 - val_loss: 0.6944 - val_auc: 0.5102
Epoch 7/30
276/276 [==============================] - 3s 11ms/step - loss: 0.7006 - auc: 0.5107 - val_loss: 0.6943 - val_auc: 0.5100
Epoch 8/30
276/276 [==============================] - 3s 12ms/step - 

In [17]:
model.save(f'model.h5')

In [18]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test, steps = len(X_test)//batch_size)

print("test loss, test auc:", results)

Evaluate on test data
76/76 [==============================] - 1s 7ms/step - loss: 0.6915 - auc: 0.5304
test loss, test auc: [0.6915105581283569, 0.5303674340248108]


In [19]:
prediction = model.predict_classes(X_test).reshape(1,-1)

# Utility score

In [20]:
result_df = pd.DataFrame({'Date': df_test['date'], 'Weight': df_test['weight'], 'Resp': df_test['resp'], 'Action': prediction[0]} )
result_df['P'] = result_df['Weight']*result_df['Resp']*result_df['Action']
result_df.head()

,Date,Weight,Resp,Action,P
1493256,430,0.195581,0.032779,1,0.006411
1493257,430,11.982266,-0.004582,0,-0.000000
1493258,430,1.107787,-0.004491,1,-0.004975
1493259,430,1.312454,0.000542,1,0.000711
1493260,430,0.422074,-0.004623,1,-0.001951


In [21]:
result_groupby_days = result_df[['Date', 'P']].groupby('Date').sum().reset_index()
print(result_groupby_days.shape)
result_groupby_days.head()

(70, 2)


,Date,P
0,430,-8.400912
1,431,-11.454133
2,432,-6.186859
3,433,-8.364620
4,434,4.384512


In [22]:
p = result_groupby_days['P'].values

t = (np.sum(p)/(np.sqrt(np.sum(p**2))))*np.sqrt(250/len(p))

u = min(max(t, 0), 6) * np.sum(p)

print(f"Utility score is: {u:.3f}")

Utility score is: 2040.143


In [23]:
# result_df[['Date', 'Action']].to_csv('NN_result.csv')